In [36]:
# EDA e Visualização de Dados
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from mlxtend.frequent_patterns import apriori, association_rules



# Carregar os Dados

In [37]:
# Carregar os dados
df_transacoes = pd.read_csv('datasets/transactions_by_dept.csv', sep=',')

In [38]:
df_transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4539 entries, 0 to 4538
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   POS Txn  4539 non-null   uint64
 1   Dept     4539 non-null   object
 2   ID       4539 non-null   int64 
 3   Sales U  4539 non-null   int64 
dtypes: int64(2), object(1), uint64(1)
memory usage: 142.0+ KB


In [39]:
df_transacoes.head(10)

,POS Txn,Dept,ID,Sales U
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1
5,16120100160021008773,0983:WINE,177,4
6,16120100160021008773,0991:TOBACCO,179,2
7,16120100160021008774,0597:HEALTH AIDS,93,1
8,16120100160021008774,0604:PERSONAL CARE,100,5
9,16120100160021008775,0819:PRE-RECORDED A/V,135,1


In [40]:
df_transacoes.tail(10)

,POS Txn,Dept,ID,Sales U
4529,16120100160162000841,0857:PC PERIPH/OFFICE ACC,155,1
4530,16120100160162000841,0931:BATH,165,1
4531,16120100160162000841,0941:BEDDING,167,1
4532,16120100160162000841,0991:TOBACCO,179,1
4533,16120100160162000842,0815:AUDIO ELECTRONICS,132,1
4534,16120100160162000843,0857:PC PERIPH/OFFICE ACC,155,1
4535,16120100160162000845,0395:MENS ATHLETIC SHOES,49,1
4536,16120100160162000845,0815:AUDIO ELECTRONICS,132,1
4537,16120100160162000846,0532:AMERICAN GREETINGS,72,1
4538,16120100160221001467,0066:VENDING/AMUSEMENT MA,242,1


# EDA

In [41]:
# Renomear as colunas para facilitar o entendimento
df_transacoes.rename(columns={'POS Txn': 'ID_Transacao', 'Dept': 'Departamento',
                              'ID': 'ID_Departamento', 'Sales U': 'Qtde_Vendida'}, inplace=True)

In [42]:
df_transacoes.head(10)

,ID_Transacao,Departamento,ID_Departamento,Qtde_Vendida
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1
5,16120100160021008773,0983:WINE,177,4
6,16120100160021008773,0991:TOBACCO,179,2
7,16120100160021008774,0597:HEALTH AIDS,93,1
8,16120100160021008774,0604:PERSONAL CARE,100,5
9,16120100160021008775,0819:PRE-RECORDED A/V,135,1


In [43]:
# Quantidade de departamentos
df_transacoes['Departamento'].nunique()

160

In [44]:
# Quantidade de ID departamentos
df_transacoes['ID_Departamento'].nunique()

160

In [47]:
# Quantidade de Transacoes
df_transacoes['ID_Transacao'].nunique()

2064

In [53]:
# Um departamento aparece mais de 1x em uma transação?
len(df_transacoes.groupby(['ID_Transacao', 'Departamento'])) != len(df_transacoes)

False

In [56]:
# Checar se há quantidades negativas e 0
len(df_transacoes[df_transacoes['Qtde_Vendida'] <= 0])

149

In [57]:
# Remover as linhas com quantidade menor ou igual a 0
df_transacoes = df_transacoes[df_transacoes['Qtde_Vendida'] > 0]

In [59]:
# Quantidade de departamentos, ID departamentos, Transacoes
df_transacoes['Departamento'].nunique(), df_transacoes['ID_Departamento'].nunique(), df_transacoes['ID_Transacao'].nunique()

(156, 156, 1943)

In [65]:
# Quais departamentos tem maior volume de transações e maior quantidade de unidades vendidas?
# Parte 1: Volume de transacoes

# Quantidade de transcaoes unicas onde cada departamento aparece
contagem_transacoes_por_departamento = df_transacoes.value_counts('Departamento')

# Plot
fig_volume_transacoes = px.bar(contagem_transacoes_por_departamento, color=contagem_transacoes_por_departamento, orientation='h')
fig_volume_transacoes.update_layout(showlegend=False)
fig_volume_transacoes.show()

In [70]:
# Quais top 10 departamentos tem maior volume de transações e maior quantidade de unidades vendidas?
# Parte 1: Volume de transacoes

# Quantidade de transcaoes unicas onde cada departamento aparece
contagem_transacoes_por_departamento_top10 = df_transacoes.value_counts('Departamento').head(10)

# Plot
fig_volume_transacoes_top10 = px.bar(contagem_transacoes_por_departamento_top10, color=contagem_transacoes_por_departamento_top10, orientation='h')
fig_volume_transacoes_top10.update_layout(showlegend=False)
fig_volume_transacoes_top10.show()

In [ ]:
# Qual departamentos tem maior volume de transações e maior quantidade de unidades vendidas?
# Parte 2: Quantidade de unidades vendidas

# Soma de Unidades Vendidas das transações onde cada departamento aparece
soma_quantidades_por_departamentos = df_transacoes.groupby('Departamento')['Qtde_Vendida'].sum().sort_values(ascending=False)

# Plot
fig_unidades_vendidas = px.bar(soma_quantidades_por_departamentos, color=soma_quantidades_por_departamentos, orientation='h')
fig_unidades_vendidas.update_layout(showlegend=False)
fig_unidades_vendidas.show()

In [74]:
# Qual o Top 10 departamentos tem maior volume de transações e maior quantidade de unidades vendidas?

# Parte 2: Quantidade de unidades vendidas
# Soma de Unidades Vendidas das transações onde cada departamento aparece
soma_quantidades_por_departamentos_top10 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].sum().sort_values(ascending=False).head(10)

# Plot
fig_unidades_vendidas_top10 = px.bar(soma_quantidades_por_departamentos_top10, color=soma_quantidades_por_departamentos_top10, orientation='h')
fig_unidades_vendidas_top10.update_layout(showlegend=False)
fig_unidades_vendidas_top10.show()

In [76]:
# Unificar dois gráficos (Top 10), para mostrar lado a lado

# Criar subplots
fig_pergunta1 = make_subplots(rows=1, cols=2, subplot_titles=("Volume de Transações", "Unidades Vendidas"))

# Adicionar os gráficos aos subplots
for trace in fig_volume_transacoes_top10['data']:
    fig_pergunta1.add_trace(trace, row=1, col=1)

for trace in fig_unidades_vendidas_top10['data']:
    fig_pergunta1.add_trace(trace, row=1, col=2)

fig_pergunta1.update_layout(height=800, width=1000, title_text="Top 10 Departamentos", showlegend=False)
fig_pergunta1.show()


In [77]:
# Quais departamentos tem a maior variação nas unidades vendidas por transação? Top 20

# Calcular o desvio padrão das quantidades vendidas, agrupando por departamento

desvio_padrao_quantidades_por_departamentos_top20 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].std().sort_values(ascending=False).head(20)

# Plot
fig_variacao_unidades_vendidas_top20 = px.bar(desvio_padrao_quantidades_por_departamentos_top20, color=desvio_padrao_quantidades_por_departamentos_top20, orientation='h')
fig_variacao_unidades_vendidas_top20.update_layout(showlegend=False)
fig_variacao_unidades_vendidas_top20.show()